# Flats [Feb 2019] - Red array

This reduction includes the data taken before the FIFI-LS series in Feb-Mar 2019.

The method followed to obtain the flats is the following:

1. First, for each spaxel $i$, I obtain a median spectrum: $f_i (\lambda) $ using all the spexels $i, j$ with $j=0,15$.

2. Then, I obtain a median spectrum from all the spaxels  $f (\lambda)$ assuming a series of constant factors to scale the different spaxel spectra to a median spectrum. I call these factors spatial flats ($flat_{spat; i}$).

3. At this point, I go back to the original spexels and obtain the spectral relative response (or spectral flat)
using the ratios:   $flat_{spec; i, j} (\lambda) = f (\lambda) / f_{i, j} (\lambda)$.

4. At the end, I merge the spaxel spectral flat and spexels spectral flat to obtain a merged one.

In this way, I obtain a cube of spectral responses and an array of spatial flats.
The spectral responses should be computable from laboratory values. The spatial flats depend on the illumination of 
the array and, in principle, can vary between laboratory and sky.
Also, I obtain a spatial flat for each calibration series and only one spectral flat common to all the series.


In [ ]:
# Used if in non-interactive mode to skip tests
Test = False

# Red 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from fifipy.io import readSlopeFits
from fifipy.spectra import computeMedianSpatialSpectra, computeSplineFits
import numpy as np

# Read slopes and compute median spectra
caldir1 = '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/calsource_flat_08_2018/'
files1 = ['D130R_1.fits','D130R_2.fits','D105R_1.fits','D105R_2.fits']
date1 = ['8/18'] * len(files1)
dichroic1 = [130,130,105,105]
caldir2 = '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/calsource_flat_02_2018/'
files2 =  ['D105R_1.fits','D105R_2.fits','D105R_3.fits','D105R_5.fits',
           'D105R_6.fits','D130R_1.fits','D130R_2.fits','D130R_3.fits']
date2 = ['2/18'] * len(files2)
dichroic2 = [105,105,105,105,105,130,130,130]
caldir3 = '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/calsource_flat_03_2017/'
files3 =  ['D105R_1.fits','D130R_1.fits']
dichroic3 = [105,130]
date3 = ['3/17'] * len(files3)
caldir = '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/calsource_flat_10_2018/'
files = ['D105R.fits','D105R_1.fits','D105R_2.fits','D130R.fits','D130R_1.fits']
dichroic = [105,105,105,130,130]
date = ['10/18'] * len(files)
caldir4 =  '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/calsource_flat_02_2019/'
files4 = ['D105R_1.fits','D105R_2.fits','D130R_1.fits','D130R_2.fits']
date4 = ['2/19'] * len(files4)
dichroic4 = [105,105,130,130]

# All
filess = [files,files1,files2,files3,files4]
caldirs = [caldir,caldir1,caldir2,caldir3,caldir4]
signs = [1,-1,-1,1,-1]
dichroics = np.concatenate([dichroic, dichroic1, dichroic2, dichroic3, dichroic4])
dates = np.concatenate([date, date1, date2, date3, date4])

wmin = 100
wmax = 200
for files, caldir in zip(filess, caldirs):
    for file in files:
        g, w, dw, s = readSlopeFits(caldir, file)
        wmin_ = np.nanmin(w); wmax_ = np.nanmax(w)
        if wmin_ < wmin: wmin = wmin_
        if wmax_ > wmax: wmax = wmax_
    
waves = []
spectra = []
for files, caldir, sign in zip(filess, caldirs, signs):
    for file in files:
        g, w, dw, s = readSlopeFits(caldir, file)
        wave, spectrum = computeSplineFits(w,dw,s,'R', wmin, wmax)
        waves.append(wave)
        spectra.append(sign*spectrum)
 
spectra = np.array(spectra)
print(np.shape(spectra))

spatspectra = []
for spectrum in spectra:
    spatspectrum = computeMedianSpatialSpectra(spectrum)
    spatspectra.append(spatspectrum)

spatspectra = np.array(spatspectra)

Reading previous flats produced by Christian Fischer

In [ ]:
# Grab Christian's flats
chrisdir = '/Users/dfadda/sofia/FIFI-LS/SpectralFlats/Christian/flats/'
wch = np.arange(119,206,1)
nw_ = np.size(wch)
f105 = np.ones((nw_,25,16))
f130 = np.ones((nw_,25,16))
ff105 = np.ones((nw_,25))
ff130 = np.ones((nw_,25))

nw = range(nw_)
for w,i in zip(wch,nw):
    wstr = str(w)
    try:
        f = np.loadtxt(chrisdir+wstr+'/flat.txt')
        f105[i,:,:] = f
        f = np.loadtxt(chrisdir+wstr+'/flatfield.txt')
        ff105[i,:] = f
    except:
        pass
    try:
        f = np.loadtxt(chrisdir+wstr+'_D130/flat.txt')
        f130[i,:,:] = f
        f = np.loadtxt(chrisdir+wstr+'_D130/flatfield.txt')
        ff130[i,:] = f
    except:
        pass

## Step 1

Find relative factors for different series.


In [ ]:
# Plot of  median spaxel spectra over median total spectra (except bad illuminated column)
from fifipy.stats import biweightLocation

s1 = np.ones(25)
s2 = np.ones(25)
s3 = np.ones(25)
s4 = np.ones(25)
s5 = np.ones(25)

mask = [0,1,2,3,5,6,7,8,10,11,12,13,15,16,17,18,20,21,22,23]
#mask = range(25)
for ispax in range(25):
    fig,ax = plt.subplots(figsize=(16,8))
    m1 = []
    m2 = []
    m3 = []
    m4 = []
    m5 = []
    for w, s, d, dic in zip(waves, spatspectra, dates, dichroics):
        meds = biweightLocation(s[mask,:], axis = 0)
        idw = (w > 130) & (w < 170)
        m = biweightLocation(s[ispax,idw]/meds[idw])    
        if d == '10/18':
            color='skyblue'
            if dic == 105:
                m1.append(m)
        elif d == '8/18':
            color='red'
            if dic == 105:
                m2.append(m)
        elif d == '2/18':
            color='green'
            if dic == 105:
                m3.append(m)
        elif d == '3/17':
            color='pink'
            if dic == 105:
                m4.append(m)
        elif d == '2/19':
            color='purple'
            if dic == 105:
                m5.append(m)
        else:
            color='black'
        if dic == 105:
            line = '-'
        else:
            line = '--'
        #if dic == 105:
        ax.plot(w, s[ispax,:]/meds, line, label = d, color=color)
    ax.plot(wch, ff105[:,ispax],color='lime',label='Christian 105',linewidth=2)
    ax.plot(wch, ff130[:,ispax],color='orange',label='Christian 130',linewidth=2)
    ax.legend()
    ax.set_title(str(ispax))
    for m, col, s in zip([m1,m2,m3,m4,m5],['skyblue','red','green','pink','purple'],[s1,s2,s3,s4,s5]):
        m = np.array(m)
        medm = np.nanmedian(m)
        ax.set_ylim(medm-0.5, medm+0.5)
        ax.plot(w, np.full(len(w), medm), color=col)
        s[ispax] = medm
    plt.show()


## Step 2

Find relative response (to the median spectrum) for each spaxel normalized to the flats found in step 1.
In this way we can use all the data. 

The two dichroics will be treated separately.

In [ ]:
from fifipy.stats import biweightLocation

mask = [1,2,3,5,6,7,8,10,11,12,13,15,16,17,18,20,21,22,23]
spaxelSpectralFlatA = []
spaxelSpectralFlatB = []
for ispax in range(25):
    fig,ax = plt.subplots(figsize=(16,8))
    specA = []
    specB = []
    for w, s, d, dic in zip(waves, spatspectra, dates, dichroics):
        meds = biweightLocation(s[mask,:], axis = 0)
        if d == '10/18':
            color='skyblue'
            ss = s1
        elif d == '8/18':
            color='red'
            ss = s2
        elif d == '2/18':
            color='green'
            ss = s3
        elif d == '3/17':
            color='pink'
            ss = s4
        elif d == '2/19':
            color='purple'
            ss = s5
        else:
            color='black'
        if dic == 105:
            line = '-'
        else:
            line = '--'
        if dic == 105:
            specA.append(s[ispax,:]/meds/ss[ispax])
        else:
            specB.append(s[ispax,:]/meds/ss[ispax])
        ax.plot(w, s[ispax,:]/meds/ss[ispax], line, label = d, color=color,alpha=0.4)
    specA = np.array(specA)
    specB = np.array(specB)
    ssf = biweightLocation(specA, axis=0)
    spaxelSpectralFlatA.append(ssf)
    ax.plot(w, ssf, color='black',label='biweight mean')
    ssf = biweightLocation(specB, axis=0)
    spaxelSpectralFlatB.append(ssf)
    ax.plot(w, ssf, '--', color='black',label='biweight mean')
    ax.plot(wch, ff105[:,ispax]/s4[ispax],color='orange',label='Christian 105',linewidth=2)
    ax.plot(wch, ff130[:,ispax]/s4[ispax],'--',color='orange',label='Christian 130',linewidth=2)

    ax.legend()
    ax.set_title(str(ispax + 1))
    ax.set_ylim([0,2])
    ax.grid()
    plt.show()


## Step 3

Correcting the original spectra with spaxel spectral flats, recompute median spectra per spaxel,
and compute the spectral flats per spexel.

At the end, we can merge the two components or leave them separated.
In the future, we should have the possibility of adding a 5x5 real spatial flat 
which can be derived from sky or moon measurements.